<a href="https://colab.research.google.com/github/jackjeanab/LangChain4Gemini/blob/main/4_6%E5%AF%A6%E4%BD%9CConditionalChain%E6%A9%9F%E5%88%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install --upgrade --quiet  langchain-google-genai

In [45]:
# 引入Chain模組
from langchain.chains import LLMChain

# 引入Gemini LLM模組
from langchain_google_genai import ChatGoogleGenerativeAI

# 引入prompt模組
from langchain_core.prompts import PromptTemplate

In [46]:
import os
from google.colab import userdata

# Get the API key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Initialize the Gemini Pro model
# Use "gemini-1.5-pro-latest" or "gemini-1.5-flash-latest" model for better instruction following
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0.5)

In [47]:
# 定義情緒分析的提示樣板
sentiment_analysis_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="根據這段話分析情緒，並僅回答 'positive' 或 'negative'：'{user_input}'"
)
# 建立情緒分析的 LLMChain
sentiment_analysis_chain = LLMChain(llm=llm, prompt=sentiment_analysis_prompt)

In [48]:
# 負面情緒應對的 PromptTemplate
negative_response_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="使用者說了這段話：'{user_input}'。請給出一段安撫的回應。"
)
negative_response_chain = LLMChain(llm=llm, prompt=negative_response_prompt)

In [49]:
# 正面情緒應對的 PromptTemplate
positive_response_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="使用者說了這段話：'{user_input}'。請給出一段正向互動的回應。"
)
positive_response_chain = LLMChain(llm=llm, prompt=positive_response_prompt)

In [50]:
def execute_conditional_chain(user_input):
    # 第一步：使用 LLM 來分析情緒
    sentiment_result = sentiment_analysis_chain.run({"user_input": user_input})

    # 第二步：根據情緒結果選擇要執行的chain
    if sentiment_result.strip().lower() == "negative":
        # 如果情緒為負面，執行負面應對chain
        return negative_response_chain.invoke({"user_input": user_input})
    else:
        # 如果情緒為正面，執行正面應對鏈結
        return positive_response_chain.invoke({"user_input": user_input})

In [51]:
# 執行 Conditional Chain
result = execute_conditional_chain("我對於你們的服務感到普通，服務人員表現普通。")

print(result["text"])

謝謝您提供寶貴的意見。我們很重視您的回饋，並對於您這次的體驗感到抱歉，似乎沒有達到您的期望。我們理解「普通」代表還有進步空間，希望能更了解哪些方面讓您覺得不夠滿意。  方便的話，是否能提供更多細節，例如哪些服務環節或服務人員的哪些方面讓您覺得普通？您的意見將有助於我們改進服務品質，提供更好的體驗。  我們期盼未來有機會再次為您服務，並展現我們提升後的服務。
